In [4]:
# ===================================================================
# Step 1: Setup and Imports
# ===================================================================
import pandas as pd
from datetime import datetime
import os
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.metrics import classification_report, roc_auc_score, accuracy_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
import joblib # Added import for saving model

print("Step 1: All libraries imported successfully.")

# ===================================================================
# Step 2: Load Raw Data
# ===================================================================
print("\nStep 2: Loading raw data...")

# --- PATHS CORRECTED ---
# Paths are now relative to the 'notebooks' folder
path_to_raw_data = '../../data_collection/synthea/output/csv/'
path_to_processed_data = '../data/processed/'
# --- END CORRECTION ---

os.makedirs(path_to_processed_data, exist_ok=True)

try:
    patients_df = pd.read_csv(path_to_raw_data + 'patients.csv')
    conditions_df = pd.read_csv(path_to_raw_data + 'conditions.csv')
    observations_df = pd.read_csv(path_to_raw_data + 'observations.csv')
    print("-> Raw data files loaded successfully.")
except FileNotFoundError as e:
    print(f"\nERROR: Could not find raw data files. Checked location: {path_to_raw_data}")
    print(f"Full error: {e}")
    # exit() # Commented out exit() to allow notebook to run partially
except Exception as e:
    print(f"\nAn unexpected error occurred loading data: {e}")
    # exit()

# ===================================================================
# Step 3: Create the Target Variable (Hypertension Label)
# ===================================================================
print("\nStep 3: Creating the target variable (HYPERTENSION)...")

if 'conditions_df' in locals() and 'patients_df' in locals():
    hypertension_conditions = conditions_df[conditions_df['DESCRIPTION'].str.contains('Hypertension', case=False)]
    patients_with_hypertension = hypertension_conditions['PATIENT'].unique()
    patients_df['HYPERTENSION'] = patients_df['Id'].isin(patients_with_hypertension).astype(int)
    print(f"-> Identified {len(patients_with_hypertension)} patients with a hypertension diagnosis.")
else:
    print("-> Skipping Step 3, raw data not loaded.")

# ===================================================================
# Step 4: Feature Engineering
# ===================================================================
print("\nStep 4: Engineering features...")

def get_latest_observation(df, code, new_column_name):
    temp_df = df[df['CODE'] == code].copy()
    temp_df['DATE'] = pd.to_datetime(temp_df['DATE'])
    temp_df = temp_df.sort_values(by=['PATIENT', 'DATE'])
    latest_obs = temp_df.drop_duplicates(subset='PATIENT', keep='last')
    latest_obs = latest_obs[['PATIENT', 'VALUE']].rename(columns={'VALUE': new_column_name})
    return latest_obs

if 'patients_df' in locals() and 'observations_df' in locals():
    patients_df['BIRTHDATE'] = pd.to_datetime(patients_df['BIRTHDATE'])
    current_year = datetime.now().year
    patients_df['AGE'] = current_year - patients_df['BIRTHDATE'].dt.year
    latest_bmi = get_latest_observation(observations_df, '39156-5', 'BMI')
    latest_systolic = get_latest_observation(observations_df, '8480-6', 'SYSTOLIC_BP')
    latest_diastolic = get_latest_observation(observations_df, '8462-4', 'DIASTOLIC_BP')
    latest_cholesterol = get_latest_observation(observations_df, '2093-3', 'TOTAL_CHOLESTEROL')
    latest_smoking = get_latest_observation(observations_df, '72166-2', 'SMOKING_STATUS')
    latest_smoking['SMOKING_STATUS'] = latest_smoking['SMOKING_STATUS'].replace({
        'Never smoker': 'non-smoker',
        'Former smoker': 'former-smoker',
        'Current every day smoker': 'smoker'
    }).fillna('unknown')
    print("-> AGE, BMI, Blood Pressure, Cholesterol, and Smoking Status features created.")
else:
    print("-> Skipping Step 4, raw data not loaded.")

# ===================================================================
# Step 5: Combine, Clean, and Save Processed Data (FIXED)
# ===================================================================
print("\nStep 5: Combining and cleaning data...")

if 'patients_df' in locals():
    model_df = patients_df[['Id', 'AGE', 'HYPERTENSION']]
    features_to_merge = [latest_bmi, latest_systolic, latest_diastolic, latest_cholesterol, latest_smoking]
    for feature_df in features_to_merge:
        model_df = pd.merge(model_df, feature_df, left_on='Id', right_on='PATIENT', how='left')
        if 'PATIENT' in model_df.columns:
            model_df = model_df.drop(columns=['PATIENT'])

    numeric_features = ['AGE', 'BMI', 'SYSTOLIC_BP', 'DIASTOLIC_BP', 'TOTAL_CHOLESTEROL']
    categorical_features = ['SMOKING_STATUS']
    feature_columns = numeric_features + categorical_features

    # --- FIX: Add .copy() here to create an independent DataFrame and prevent the warning ---
    final_df = model_df[feature_columns + ['HYPERTENSION']].copy()

    # Convert to numeric BEFORE imputation
    for col in numeric_features:
        final_df[col] = pd.to_numeric(final_df[col], errors='coerce')

    # Now we can safely impute missing values
    for col in numeric_features:
        median_val = final_df[col].median()
        final_df[col] = final_df[col].fillna(median_val)
    for col in categorical_features:
        mode_val = final_df[col].mode()[0]
        final_df[col] = final_df[col].fillna(mode_val)

    print("-> Missing values handled and data cleaned.")

    # Save the clean dataframe
    output_path = path_to_processed_data + 'cleaned_hypertension_data.csv'
    final_df.to_csv(output_path, index=False)
    print(f"-> Clean data saved to '{output_path}'")
    print(f"-> Shape of final model-ready data: {final_df.shape}")
else:
    print("-> Skipping Step 5, data not processed.")


# ===================================================================
# Step 6: Prepare for Modeling
# ===================================================================
print("\nStep 6: Preprocessing features and splitting data...")

if 'final_df' in locals():
    X = final_df[feature_columns]
    y = final_df['HYPERTENSION']

    preprocessor = ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), numeric_features),
            ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
        ])
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
    print(f"-> Training set size: {X_train.shape[0]} samples")
    print(f"-> Test set size: {X_test.shape[0]} samples")
else:
    print("-> Skipping Step 6, final_df not created.")

# ===================================================================
# Step 7: Train and Evaluate Models
# ===================================================================
print("\nStep 7: Training and evaluating models...")

if 'X_train' in locals():
    try:
        scale_pos_weight = y_train.value_counts()[0] / y_train.value_counts()[1]
    except (ZeroDivisionError, KeyError):
        scale_pos_weight = 1

    models = {
        "Logistic Regression": Pipeline(steps=[('preprocessor', preprocessor), ('classifier', LogisticRegression(random_state=42, class_weight='balanced'))]),
        "Random Forest": Pipeline(steps=[('preprocessor', preprocessor), ('classifier', RandomForestClassifier(random_state=42, class_weight='balanced'))]),
        "XGBoost": Pipeline(steps=[('preprocessor', preprocessor), ('classifier', xgb.XGBClassifier(random_state=42, scale_pos_weight=scale_pos_weight, use_label_encoder=False, eval_metric='logloss'))])
    }

    for name, pipeline in models.items():
        print(f"\n{'='*20} {name} {'='*20}")
        pipeline.fit(X_train, y_train)
        y_pred = pipeline.predict(X_test)
        y_pred_proba = pipeline.predict_proba(X_test)[:, 1]
        
        print(f"\nClassification Report:")
        print(classification_report(y_test, y_pred))
        acc_score = accuracy_score(y_test, y_pred)
        print(f"Accuracy Score: {acc_score:.4f}")
        auc_score = roc_auc_score(y_test, y_pred_proba)
        print(f"AUC-ROC Score: {auc_score:.4f}\n")
else:
    print("-> Skipping Step 7, data not split.")

# ===================================================================
# Step 8: Save the Final Model
# ===================================================================
print("\nStep 8: Saving best model to file...")

if 'models' in locals():
    # This selects the trained Random Forest model from the 'models' dictionary
    final_model = models["Random Forest"]

    # --- PATHS CORRECTED ---
    # Define the directory and create it if it doesn't exist
    model_directory = '../models/'
    # --- END CORRECTION ---
    
    os.makedirs(model_directory, exist_ok=True)

    # Define the full filename
    model_filename = model_directory + 'hypertension_model.joblib'

    # Save the model
    joblib.dump(final_model, model_filename)

    print(f"✅ Hypertension model saved successfully to {model_filename}")
else:
    print("-> Skipping Step 8, models not trained.")

print("--- End of Script ---")

Step 1: All libraries imported successfully.

Step 2: Loading raw data...
-> Raw data files loaded successfully.

Step 3: Creating the target variable (HYPERTENSION)...
-> Identified 275 patients with a hypertension diagnosis.

Step 4: Engineering features...
-> AGE, BMI, Blood Pressure, Cholesterol, and Smoking Status features created.

Step 5: Combining and cleaning data...
-> Missing values handled and data cleaned.
-> Clean data saved to '../data/processed/cleaned_hypertension_data.csv'
-> Shape of final model-ready data: (1171, 7)

Step 6: Preprocessing features and splitting data...
-> Training set size: 936 samples
-> Test set size: 235 samples

Step 7: Training and evaluating models...

==================== Logistic Regression ====================

Classification Report:
              precision    recall  f1-score   support

           0       0.95      0.68      0.79       180
           1       0.46      0.87      0.60        55

    accuracy                           0.73   

c:\Users\LENOVO\miniconda3\lib\site-packages\xgboost\training.py:199: UserWarning: [00:24:13] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)



Classification Report:
              precision    recall  f1-score   support

           0       0.83      0.82      0.82       180
           1       0.43      0.45      0.44        55

    accuracy                           0.73       235
   macro avg       0.63      0.64      0.63       235
weighted avg       0.74      0.73      0.73       235

Accuracy Score: 0.7319
AUC-ROC Score: 0.7826


Step 8: Saving best model to file...
✅ Hypertension model saved successfully to ../models/hypertension_model.joblib
--- End of Script ---
